In [ ]:
import tkinter as tk
from PIL import Image, ImageTk

root = tk.Tk()
root.overrideredirect(True)  # sin bordes
root.attributes("-topmost", True)
root.attributes("-alpha", 0.9)  # transparencia general

img = Image.open("/home/pk/Desktop/frontend/public/img/12.png")
photo = ImageTk.PhotoImage(img)

label = tk.Label(root, image=photo, bg="black")
label.image = photo   # 🔹 evita que la imagen se elimine
label.pack()

root.geometry("+100+100")  # posición
root.mainloop()



TclError: image "pyimage1" doesn't exist

In [2]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Crear un PDF tamaño A3 apaisado
# Tamaño A3 en pulgadas = (11.7, 16.5)
with PdfPages("grafico_a3_apaisado.pdf") as pdf:
    fig, ax = plt.subplots(figsize=(16.5, 11.7))  # A3 horizontal (landscape)
    
    # Crear un gráfico de ejemplo
    x = range(10)
    y = [i**2 for i in x]
    ax.plot(x, y, marker="o", linewidth=2)
    
    # Títulos y etiquetas
    ax.set_title("Ejemplo de gráfico en PDF tamaño A3 apaisado", fontsize=18)
    ax.set_xlabel("Eje X")
    ax.set_ylabel("Eje Y (valores al cuadrado)")
    ax.grid(True)
    
    # Guardar la figura en el PDF
    pdf.savefig(fig)
    plt.close(fig)

print("✅ PDF generado: grafico_a3_apaisado.pdf")


ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# ---------------- CONFIGURACIÓN ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Título del documento (para el pie de página)
DOCUMENT_TITLE = "DOCUMENTO FINAL CON SECCIONES"

# Información portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- FUNCIÓN PARA AGREGAR PIE DE PÁGINA ----------------
def add_footer_overlay(page, page_num):
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    width, height = A4

    rect_height = 25
    y_rect = 20 - offset_y  # posición base del pie

    # Fondo blanco para cubrir el pie anterior
    can.setFillColorRGB(1, 1, 1)
    can.rect(0, 0, width, y_rect + rect_height, fill=1, stroke=0)

    # Línea gris divisoria
    can.setStrokeColorRGB(0.8, 0.8, 0.8)
    can.setLineWidth(0.5)
    can.line(40, y_rect + rect_height - 5, width - 40, y_rect + rect_height - 5)

    # Texto del pie: izquierda y derecha
    can.setFont("Helvetica", 9)
    can.setFillColorRGB(0.4, 0.4, 0.4)  # Gris medio

    # Izquierda → título + fecha
    left_text = f"{DOCUMENT_TITLE}  |  {date_str}"
    can.drawString(40, 20 - offset_y, left_text)

    # Derecha → número de página
    can.drawRightString(width - 40, 20 - offset_y, str(page_num))

    can.save()
    packet.seek(0)
    overlay_pdf = PdfReader(packet)
    overlay_page = overlay_pdf.pages[0]
    page.merge_page(overlay_page)
    return page



# ---------------- RECOPILAR PDFs ----------------
structure = {}

for folder1 in sorted(os.listdir(main_folder)):
    folder1_path = os.path.join(main_folder, folder1)
    if not os.path.isdir(folder1_path):
        continue
    for folder2 in sorted(os.listdir(folder1_path)):
        folder2_path = os.path.join(folder1_path, folder2)
        if not os.path.isdir(folder2_path):
            continue
        pdfs = [os.path.join(folder2_path, f)
                for f in sorted(os.listdir(folder2_path))
                if f.lower().endswith(".pdf")]
        if pdfs:
            structure[f"{folder1}/{folder2}"] = pdfs

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100

can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- PROCESAR PDFs Y CREAR INDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for section, pdf_list in structure.items():
    section_name = section.split('/')[-1].replace("_", " ")
    index_data.append((f"{section_name}", page_counter))

    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section_name}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    page = section_pdf.pages[0]
    page = add_footer_overlay(page, page_counter)
    all_pages.append(page)
    page_counter += 1

    # PDFs de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path).replace(".pdf", "").replace("_", " ")
        index_data.append((f"  {subsec_name}", page_counter))
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            page = add_footer_overlay(page, page_counter)
            all_pages.append(page)
            page_counter += 1

# ---------------- CREAR INDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

styleSectionBold = ParagraphStyle(
    'SectionBold',
    parent=styles['Normal'],
    fontName='Helvetica-Bold',
    fontSize=12,
    leftIndent=10,
    spaceAfter=2
)

table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    if not name.startswith("  "):
        p = Paragraph(name, styleSectionBold)
    else:
        p = Paragraph(name, styleN)
    table_data.append([p, str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)
index_page = index_pdf.pages[0]
index_page = add_footer_overlay(index_page, 2)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_page)

page_num = 3
for page in all_pages:
    page = add_footer_overlay(page, page_num)
    final_pdf.add_page(page)
    page_num += 1

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"✅ PDF final creado con pie personalizado: {output_pdf}")


✅ PDF final creado con pie personalizado: /home/pk/Desktop/vatiaco/public/routers/PDF_final_secciones.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# Carpeta principal
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Recopilar PDFs organizados por segundo nivel
structure = {}
for chapter in sorted(os.listdir(main_folder)):
    chapter_path = os.path.join(main_folder, chapter)
    if os.path.isdir(chapter_path):
        sections = {}
        for section in sorted(os.listdir(chapter_path)):
            section_path = os.path.join(chapter_path, section)
            if os.path.isdir(section_path):
                pdfs = [os.path.join(section_path, f)
                        for f in sorted(os.listdir(section_path))
                        if f.lower().endswith(".pdf")]
                if pdfs:
                    sections[section] = pdfs
        if sections:
            structure[chapter] = sections

# Crear PDF final
final_pdf = PdfWriter()
width, height = A4
index_data = []
page_counter = 1

# Portada
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
can.setFont("Helvetica-Bold", 24)
can.drawCentredString(width/2, height/2, "DOCUMENTO FINAL CON SECCIONES")
can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)
final_pdf.add_page(cover_pdf.pages[0])
page_counter += 1

# ---------------- PROCESAR PDFs ----------------
for chapter, sections in structure.items():
    chapter_name_clean = chapter.replace("_", " ")
    index_data.append((chapter_name_clean, None))

    # Página de capítulo
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 22)
    can.drawString(50, height - 100, f"Capítulo: {chapter_name_clean}")
    can.showPage()
    can.save()
    packet.seek(0)
    chapter_pdf = PdfReader(packet)
    final_pdf.add_page(chapter_pdf.pages[0])
    page_counter += 1

    for section, pdf_list in sections.items():
        section_name_clean = section.replace("_", " ")
        index_data.append(("    " + section_name_clean, page_counter))

        # Página de sección
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=A4)
        can.setFont("Helvetica-Bold", 18)
        can.drawString(50, height - 100, f"Sección: {section_name_clean}")
        can.showPage()
        can.save()
        packet.seek(0)
        section_pdf = PdfReader(packet)
        final_pdf.add_page(section_pdf.pages[0])
        page_counter += 1

        for pdf_path in pdf_list:
            doc_name_clean = os.path.basename(pdf_path).replace(".pdf", "").replace("_", " ")
            index_data.append(("        " + doc_name_clean, page_counter))

            reader = PdfReader(pdf_path)
            for page in reader.pages:
                # Crear overlay con pie de página
                packet = BytesIO()
                can = canvas.Canvas(packet, pagesize=A4)
                now_str = datetime.now().strftime("%Y-%m-%d")
                
                # Pie de página
                can.setFont("Helvetica", 9)
                can.setFillColorRGB(0, 0, 0)
                can.drawString(40, 30 - offset_y, doc_name_clean[:70])  # Nombre documento
                can.drawCentredString(width / 2, 30 - offset_y, now_str)  # Fecha
                can.drawRightString(width - 40, 30 - offset_y, str(page_counter))  # Nº página
                can.save()

                # Superponer sobre la página
                packet.seek(0)
                overlay_pdf = PdfReader(packet)
                overlay_page = overlay_pdf.pages[0]
                page.merge_page(overlay_page)

                final_pdf.add_page(page)
                page_counter += 1

# ---------------- CREAR ÍNDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
styles = getSampleStyleSheet()
styleN = styles['Normal']
styleBold = ParagraphStyle('Bold', parent=styleN, fontName='Helvetica-Bold')
styleIndented = ParagraphStyle('Indented', parent=styleN, leftIndent=20)
styleIndentedBold = ParagraphStyle('IndentedBold', parent=styleBold, leftIndent=20)
styleMoreIndented = ParagraphStyle('MoreIndented', parent=styleN, leftIndent=40)

elements = []
elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo / Sección / Documento", "Página"]]
for name, page_no in index_data:
    if page_no is None:
        table_data.append([Paragraph(name, styleBold), ""])
    elif name.startswith("    "):  # Sección
        table_data.append([Paragraph(name.strip(), styleIndentedBold), str(page_no)])
    elif name.startswith("        "):  # Documento
        table_data.append([Paragraph(name.strip(), styleMoreIndented), str(page_no)])
    else:  # Capítulo
        table_data.append([Paragraph(name, styleBold), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.whitesmoke),
    ('BOX', (0, 0), (-1, -1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0, 0), (-1, -1), 11),
    ('LEFTPADDING', (0, 0), (-1, -1), 5),
    ('RIGHTPADDING', (0, 0), (-1, -1), 5),
    ('ALIGN', (1, 1), (1, -1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)
final_pdf.add_page(index_pdf.pages[0])

# ---------------- GUARDAR PDF FINAL ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"✅ PDF final creado con éxito: {output_pdf}")


✅ PDF final creado con éxito: /home/pk/Desktop/vatiaco/public/routers/PDF_final_secciones.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# ---------------- CONFIG ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_triple_nivel.pdf")
offset_y = 14.17  # 0.5 cm

# Portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs (3 niveles) ----------------
structure = {}
for chapter_folder in sorted(os.listdir(main_folder)):
    chapter_path = os.path.join(main_folder, chapter_folder)
    if not os.path.isdir(chapter_path):
        continue
    structure[chapter_folder] = {}
    for section_folder in sorted(os.listdir(chapter_path)):
        section_path = os.path.join(chapter_path, section_folder)
        if not os.path.isdir(section_path):
            continue
        pdfs = [os.path.join(section_path, f) for f in sorted(os.listdir(section_path)) if f.lower().endswith(".pdf")]
        if pdfs:
            structure[chapter_folder][section_folder] = pdfs

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100
can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50
can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50
can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")
can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- ESTILOS PARA ÍNDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']
styleChapterBold = ParagraphStyle('ChapterBold', parent=styleN, fontName='Helvetica-Bold', fontSize=14, leftIndent=0, spaceAfter=4)
styleSectionBold = ParagraphStyle('SectionBold', parent=styleN, fontName='Helvetica-Bold', fontSize=12, leftIndent=10, spaceAfter=2)
styleSubsection = ParagraphStyle('Subsection', parent=styleN, fontSize=12, leftIndent=20, spaceAfter=1)

# ---------------- PROCESAR PDFs ----------------
# ---------------- PROCESAR PDFs ----------------
index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for chapter, sections in structure.items():
    chapter_name_clean = chapter.replace("_", " ")  # Reemplazar guiones bajos
    index_data.append((chapter_name_clean, None))  # número de página opcional

    # Página de capítulo
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Capítulo: {chapter_name_clean}")
    can.showPage()
    can.save()
    packet.seek(0)
    chapter_pdf = PdfReader(packet)
    all_pages.append(chapter_pdf.pages[0])
    page_counter += 1

    for section, pdf_list in sections.items():
        section_name_clean = section.replace("_", " ")
        index_data.append((section_name_clean, page_counter))

        # Página de sección
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=A4)
        can.setFont("Helvetica-Bold", 18)
        can.drawString(50, height - 100, f"Sección: {section_name_clean}")
        can.showPage()
        can.save()
        packet.seek(0)
        section_pdf = PdfReader(packet)
        all_pages.append(section_pdf.pages[0])
        page_counter += 1

        for pdf_path in pdf_list:
            subsec_name_clean = os.path.basename(pdf_path).replace(".pdf","").replace("_"," ")
            index_data.append((f"  {subsec_name_clean}", page_counter))
            reader = PdfReader(pdf_path)
            for page in reader.pages:
                # Overlay número de página
                packet = BytesIO()
                can = canvas.Canvas(packet, pagesize=A4)
                rect_width, rect_height = 50, 20
                x_pos = width - 40 - rect_width/2
                y_pos = 30 - rect_height/2 - offset_y
                can.setFillColorRGB(1,1,1)
                can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
                can.setFillColorRGB(0,0,0)
                can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
                can.save()
                packet.seek(0)
                overlay_pdf = PdfReader(packet)
                overlay_page = overlay_pdf.pages[0]
                page.merge_page(overlay_page)
                all_pages.append(page)
                page_counter += 1


# ---------------- CREAR ÍNDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []
elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    if page_no is None:
        p = Paragraph(name, styleChapterBold)
        page_no_str = ""
    elif not name.startswith("  "):  # Sección
        p = Paragraph(name, styleSectionBold)
        page_no_str = str(page_no)
    else:
        p = Paragraph(name, styleSubsection)
        page_no_str = str(page_no)
    table_data.append([p, page_no_str])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]
table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)
packet.seek(0)
index_pdf = PdfReader(packet)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_pdf.pages[0])
for page in all_pages:
    final_pdf.add_page(page)

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con triple jerarquía creado: {output_pdf}")


PDF final con triple jerarquía creado: /home/pk/Desktop/vatiaco/public/routers/PDF_final_triple_nivel.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# ---------------- CONFIG ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_triple_nivel.pdf")
offset_y = 14.17  # 0.5 cm

# Portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs (3 niveles) ----------------
structure = {}
for chapter_folder in sorted(os.listdir(main_folder)):
    chapter_path = os.path.join(main_folder, chapter_folder)
    if not os.path.isdir(chapter_path):
        continue
    structure[chapter_folder] = {}
    for section_folder in sorted(os.listdir(chapter_path)):
        section_path = os.path.join(chapter_path, section_folder)
        if not os.path.isdir(section_path):
            continue
        pdfs = [os.path.join(section_path, f) for f in sorted(os.listdir(section_path)) if f.lower().endswith(".pdf")]
        if pdfs:
            structure[chapter_folder][section_folder] = pdfs

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100
can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50
can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50
can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")
can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- ESTILOS PARA ÍNDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']
styleChapterBold = ParagraphStyle('ChapterBold', parent=styleN, fontName='Helvetica-Bold', fontSize=14, leftIndent=0, spaceAfter=4)
styleSectionBold = ParagraphStyle('SectionBold', parent=styleN, fontName='Helvetica-Bold', fontSize=12, leftIndent=10, spaceAfter=2)
styleSubsection = ParagraphStyle('Subsection', parent=styleN, fontSize=12, leftIndent=20, spaceAfter=1)

# ---------------- PROCESAR PDFs ----------------
index_data = []
all_pages = []
page_counter = 2  # Portada = 1

# Mapear páginas para índice
pdf_page_map = {}

for chapter, sections in structure.items():
    # Capítulo
    index_data.append((chapter, None))  # número de página opcional
    # Página de capítulo
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Capítulo: {chapter}")
    can.showPage()
    can.save()
    packet.seek(0)
    chapter_pdf = PdfReader(packet)
    all_pages.append(chapter_pdf.pages[0])
    page_counter += 1

    for section, pdf_list in sections.items():
        # Sección
        index_data.append((section, page_counter))
        # Página de sección
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=A4)
        can.setFont("Helvetica-Bold", 18)
        can.drawString(50, height - 100, f"Sección: {section}")
        can.showPage()
        can.save()
        packet.seek(0)
        section_pdf = PdfReader(packet)
        all_pages.append(section_pdf.pages[0])
        page_counter += 1

        for pdf_path in pdf_list:
            subsec_name = os.path.basename(pdf_path).replace(".pdf","")
            index_data.append((f"  {subsec_name}", page_counter))
            reader = PdfReader(pdf_path)
            for page in reader.pages:
                # Overlay número de página
                packet = BytesIO()
                can = canvas.Canvas(packet, pagesize=A4)
                rect_width, rect_height = 50, 20
                x_pos = width - 40 - rect_width/2
                y_pos = 30 - rect_height/2 - offset_y
                can.setFillColorRGB(1,1,1)
                can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
                can.setFillColorRGB(0,0,0)
                can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
                can.save()
                packet.seek(0)
                overlay_pdf = PdfReader(packet)
                overlay_page = overlay_pdf.pages[0]
                page.merge_page(overlay_page)
                all_pages.append(page)
                page_counter += 1

# ---------------- CREAR ÍNDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
styles = getSampleStyleSheet()
styleN = styles['Normal']
styleBold = ParagraphStyle('Bold', parent=styleN, fontName='Helvetica-Bold')
styleIndented = ParagraphStyle('Indented', parent=styleN, leftIndent=20)
styleIndentedBold = ParagraphStyle('IndentedBold', parent=styleBold, leftIndent=20)
styleMoreIndented = ParagraphStyle('MoreIndented', parent=styleN, leftIndent=40)

elements = []
elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo / Sección / Documento", "Página"]]
for name, page_no in index_data:
    if page_no is None:
        table_data.append([Paragraph(name, styleBold), ""])
    elif name.startswith("    "):  # Sección
        table_data.append([Paragraph(name.strip(), styleIndentedBold), str(page_no)])
    elif name.startswith("        "):  # Documento
        table_data.append([Paragraph(name.strip(), styleMoreIndented), str(page_no)])
    else:  # Capítulo
        table_data.append([Paragraph(name, styleBold), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.whitesmoke),
    ('BOX', (0, 0), (-1, -1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0, 0), (-1, -1), 11),
    ('LEFTPADDING', (0, 0), (-1, -1), 5),
    ('RIGHTPADDING', (0, 0), (-1, -1), 5),
    ('ALIGN', (1, 1), (1, -1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)
index_page = index_pdf.pages[0]
index_page = add_footer_overlay(index_page, "ÍNDICE", 2)

# ---------------- REORDENAR Y GUARDAR ----------------
final_pdf_with_index = PdfWriter()
final_pdf_with_index.add_page(final_pdf.pages[0])  # portada
final_pdf_with_index.add_page(index_page)          # índice
for p in list(final_pdf.pages)[1:]:                # resto
    final_pdf_with_index.add_page(p)

with open(output_pdf, "wb") as f:
    final_pdf_with_index.write(f)

print(f"✅ PDF final creado con éxito: {output_pdf}")



✅ PDF final creado con éxito: /home/pk/Desktop/vatiaco/public/routers/PDF_final_triple_nivel.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime
from reportlab.lib.styles import ParagraphStyle

# ---------------- CONFIGURACIÓN ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Información portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs ----------------
structure = {}

# Recorrer subcarpetas de segundo nivel
for folder1 in sorted(os.listdir(main_folder)):
    folder1_path = os.path.join(main_folder, folder1)
    if not os.path.isdir(folder1_path):
        continue
    for folder2 in sorted(os.listdir(folder1_path)):
        folder2_path = os.path.join(folder1_path, folder2)
        if not os.path.isdir(folder2_path):
            continue
        pdfs = [os.path.join(folder2_path, f)
                for f in sorted(os.listdir(folder2_path))
                if f.lower().endswith(".pdf")]
        if pdfs:
            structure[f"{folder1}/{folder2}"] = pdfs

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100

can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- PROCESAR PDFs Y CREAR INDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for section, pdf_list in structure.items():
    # Agregar sección al índice
    section_name = section.split('/')[-1]  # Solo el nombre de la subcarpeta
    index_data.append((f"{section_name}", page_counter))
    
    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section_name}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    all_pages.append(section_pdf.pages[0])
    page_counter += 1

    # Cada PDF dentro de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path).replace(".pdf","")  # Quitar extensión
        index_data.append((f"  {subsec_name}", page_counter))  # Indentar como subsección
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            # Overlay número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            all_pages.append(page)
            page_counter += 1

# ---------------- CREAR INDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))
# Crear un estilo en negrita para secciones
# Estilo para secciones: negrita + indentación


# Estilos
styleChapterBold = ParagraphStyle(
    'ChapterBold',
    parent=styles['Normal'],
    fontName='Helvetica-Bold',
    fontSize=14,
    leftIndent=0,
    spaceAfter=4
)
styleSectionBold = ParagraphStyle(
    'SectionBold',
    parent=styles['Normal'],
    fontName='Helvetica-Bold',
    fontSize=12,
    leftIndent=10,
    spaceAfter=2
)
styleSubsection = styleN

# Construir tabla del índice
table_data = [["Capítulo/Subsección", "Página"]]

for chapter, sections in structure.items():
    # Capítulo (nivel 1)
    table_data.append([Paragraph(chapter, styleChapterBold), ""])
    for section, pdf_list in sections.items():
        # Sección (nivel 2)
        section_name = section.split('/')[-1]
        table_data.append([Paragraph(section_name, styleSectionBold), str(section_page_map[section])])
        for pdf_path in pdf_list:
            subsec_name = os.path.basename(pdf_path).replace(".pdf","")
            table_data.append([Paragraph("  "+subsec_name, styleSubsection), str(pdf_page_map[pdf_path])])


usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_pdf.pages[0])

for page in all_pages:
    final_pdf.add_page(page)

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con secciones y subsecciones creado: {output_pdf}")

AttributeError: 'list' object has no attribute 'items'

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime
from reportlab.lib.styles import ParagraphStyle

# ---------------- CONFIGURACIÓN ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Información portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs ----------------
structure = {}

# Recorrer subcarpetas de segundo nivel
for folder1 in sorted(os.listdir(main_folder)):
    folder1_path = os.path.join(main_folder, folder1)
    if not os.path.isdir(folder1_path):
        continue
    for folder2 in sorted(os.listdir(folder1_path)):
        folder2_path = os.path.join(folder1_path, folder2)
        if not os.path.isdir(folder2_path):
            continue
        pdfs = [os.path.join(folder2_path, f)
                for f in sorted(os.listdir(folder2_path))
                if f.lower().endswith(".pdf")]
        if pdfs:
            structure[f"{folder1}/{folder2}"] = pdfs

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100

can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- PROCESAR PDFs Y CREAR INDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for section, pdf_list in structure.items():
    # Agregar sección al índice
    section_name = section.split('/')[-1]  # Solo el nombre de la subcarpeta
    index_data.append((f"{section_name}", page_counter))
    
    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section_name}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    all_pages.append(section_pdf.pages[0])
    page_counter += 1

    # Cada PDF dentro de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path).replace(".pdf","")  # Quitar extensión
        index_data.append((f"  {subsec_name}", page_counter))  # Indentar como subsección
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            # Overlay número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            all_pages.append(page)
            page_counter += 1

# ---------------- CREAR INDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))
# Crear un estilo en negrita para secciones
# Estilo para secciones: negrita + indentación


styleSectionBold = ParagraphStyle(
    'SectionBold',
    parent=styles['Normal'],
    fontName='Helvetica-Bold',
    fontSize=12,
    leftIndent=10,  # Indentación de 10 puntos
    spaceAfter=2
)

# Construir tabla del índice
table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    if not name.startswith("  "):  # Es sección
        p = Paragraph(name, styleSectionBold)
    else:  # Es subsección
        p = Paragraph(name, styleN)
    table_data.append([p, str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_pdf.pages[0])

for page in all_pages:
    final_pdf.add_page(page)

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con secciones y subsecciones creado: {output_pdf}")


PDF final con secciones y subsecciones creado: /home/pk/Desktop/vatiaco/public/routers/PDF_final_secciones.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# ---------------- CONFIGURACIÓN ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Información portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs ----------------
structure = {}

# Primer nivel
for folder1 in sorted(os.listdir(main_folder)):
    folder1_path = os.path.join(main_folder, folder1)
    if not os.path.isdir(folder1_path):
        continue

    # Segundo nivel
    for folder2 in sorted(os.listdir(folder1_path)):
        folder2_path = os.path.join(folder1_path, folder2)
        if not os.path.isdir(folder2_path):
            continue

        pdfs = [os.path.join(folder2_path, f)
                for f in sorted(os.listdir(folder2_path))
                if f.lower().endswith(".pdf")]
        if pdfs:
            structure[f"{folder1}/{folder2}"] = pdfs


# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100

can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- PROCESAR PDFs Y CREAR INDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for section, pdf_list in structure.items():
    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    all_pages.append(section_pdf.pages[0])
    index_data.append((f"Sección: {section}", page_counter))
    page_counter += 1

    # Cada PDF dentro de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path)
        index_data.append((f"  {subsec_name}", page_counter))
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            # Overlay número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            all_pages.append(page)
            page_counter += 1

# ---------------- CREAR INDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    table_data.append([Paragraph(name, styleN), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_pdf.pages[0])

for page in all_pages:
    final_pdf.add_page(page)

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con secciones y subsecciones creado: {output_pdf}")


PDF final con secciones y subsecciones creado: /home/pk/Desktop/vatiaco/public/routers/PDF_final_secciones.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# ---------------- CONFIGURACIÓN ----------------
main_folder = "/home/pk/Desktop/frontend/public/routers"
output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Información portada
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# ---------------- RECOPILAR PDFs ----------------
structure = {}
for subfolder in sorted(os.listdir(main_folder)):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        pdfs = [f for f in sorted(os.listdir(subfolder_path)) if f.lower().endswith(".pdf")]
        if pdfs:
            structure[subfolder] = [os.path.join(subfolder_path, f) for f in pdfs]

# ---------------- CREAR PORTADA ----------------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4
y = height - 100

can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()
packet.seek(0)
cover_pdf = PdfReader(packet)

# ---------------- PROCESAR PDFs Y CREAR INDICE ----------------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
all_pages = []
page_counter = 2  # Portada = 1

for section, pdf_list in structure.items():
    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    all_pages.append(section_pdf.pages[0])
    index_data.append((f"Sección: {section}", page_counter))
    page_counter += 1

    # Cada PDF dentro de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path)
        index_data.append((f"  {subsec_name}", page_counter))
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            # Overlay número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            all_pages.append(page)
            page_counter += 1

# ---------------- CREAR INDICE ----------------
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    table_data.append([Paragraph(name, styleN), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# ---------------- CREAR PDF FINAL ----------------
final_pdf = PdfWriter()
final_pdf.add_page(cover_pdf.pages[0])
final_pdf.add_page(index_pdf.pages[0])

for page in all_pages:
    final_pdf.add_page(page)

# ---------------- GUARDAR ----------------
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con secciones y subsecciones creado: {output_pdf}")


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# Carpeta principal
main_folder = "/home/pk/Desktop/frontend/public/routers"

output_pdf = os.path.join(main_folder, "PDF_final_secciones.pdf")
offset_y = 14.17  # 0.5 cm

# Recopilar PDFs organizados por subcarpeta (solo segundo nivel)
structure = {}
for subfolder in sorted(os.listdir(main_folder)):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        pdfs = [f for f in sorted(os.listdir(subfolder_path)) if f.lower().endswith(".pdf")]
        if pdfs:
            structure[subfolder] = [os.path.join(subfolder_path, f) for f in pdfs]

# -------- CREAR PDF FINAL --------
final_pdf = PdfWriter()

# Añadir portada
final_pdf.add_page(cover_pdf.pages[0])

# Añadir índice
final_pdf.add_page(index_pdf.pages[0])

# Sobrescribir números de página y añadir PDFs
page_counter = 3  # Portada=1, índice=2
for section, pdf_list in structure.items():
    # Página de sección
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    width, height = A4
    can.setFont("Helvetica-Bold", 20)
    can.drawString(50, height - 100, f"Sección: {section}")
    can.showPage()
    can.save()
    packet.seek(0)
    section_pdf = PdfReader(packet)
    final_pdf.add_page(section_pdf.pages[0])
    page_counter += 1

    for pdf_path in pdf_list:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            # Overlay con número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            width, height = A4
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            final_pdf.add_page(page)
            page_counter += 1


    # Agregar cada PDF de la subsección
    for pdf_path in pdf_list:
        subsec_name = os.path.basename(pdf_path)
        reader = PdfReader(pdf_path)
        index_data.append((f"  {subsec_name}", page_counter))
        for page in reader.pages:
            # Overlay con número de página
            packet = BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            width, height = A4
            rect_width = 50
            rect_height = 20
            x_pos = width - 40 - rect_width/2
            y_pos = 30 - rect_height/2 - offset_y
            can.setFillColorRGB(1,1,1)
            can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
            can.setFillColorRGB(0,0,0)
            can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
            can.save()
            packet.seek(0)
            overlay_pdf = PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            final_pdf.add_page(page)
            page_counter += 1

# Crear índice como tabla
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

table_data = [["Capítulo/Subsección", "Página"]]
for name, page_no in index_data:
    table_data.append([Paragraph(name, styleN), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# Insertar índice después de portada
final_pdf.pages.insert(1, index_pdf.pages[0])

# Guardar PDF final
with open(output_pdf, "wb") as f:
    final_pdf.write(f)

print(f"PDF final con secciones y subsecciones creado: {output_pdf}")


AttributeError: '_VirtualList' object has no attribute 'insert'

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# -------- CONFIGURACIÓN --------
folder_path = "/home/pk/Desktop/frontend/public/routers"
output_path = os.path.join(folder_path, "PDF_final_con_portada_indice.pdf")
offset_y = 14.17  # 0.5 cm en puntos

# Información para la portada (normalizada para informe técnico)
report_title = "Informe Técnico de Datos"
report_subtitle = "Análisis y consolidación de documentos"
author = "Equipo de Análisis"
department = "Departamento de Tecnología"
version = "Versión 1.0"
date_str = datetime.now().strftime("%Y-%m-%d")

# -------- RECOPILAR PDFS --------
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))
pdf_files.sort()

# -------- PORTADA --------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4

# Espacios verticales
y = height - 100

# Título
can.setFont("Helvetica-Bold", 28)
can.drawCentredString(width/2, y, report_title)
y -= 50

# Subtítulo
can.setFont("Helvetica", 18)
can.drawCentredString(width/2, y, report_subtitle)
y -= 50

# Autor / equipo
can.setFont("Helvetica", 14)
can.drawCentredString(width/2, y, f"Autor: {author}")
y -= 25

# Departamento / organización
can.drawCentredString(width/2, y, f"Departamento: {department}")
y -= 25

# Fecha
can.drawCentredString(width/2, y, f"Fecha: {date_str}")
y -= 25

# Versión
can.drawCentredString(width/2, y, f"{version}")

can.showPage()
can.save()

packet.seek(0)
cover_pdf = PdfReader(packet)

# -------- CREAR INDICE COMO TABLA --------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
page_counter = 2  # Portada = 1
all_pages = []

for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    index_data.append((os.path.relpath(pdf_path, folder_path), page_counter))
    for page in reader.pages:
        all_pages.append(page)
        page_counter += 1

# Crear PDF de índice usando Platypus
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

# Encabezado
elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

# Tabla de índice
table_data = [["Capítulo", "Página"]]
for name, page_no in index_data:
    table_data.append([Paragraph(name, styleN), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('BOX', (0,0), (-1,-1), 0.25, colors.whitesmoke),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
    ('ALIGN', (1,1), (1,-1), 'RIGHT'),  # Alinear la columna de números a la derecha
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# -------- CREAR PDF FINAL --------
final_pdf = PdfWriter()

# Añadir portada
final_pdf.add_page(cover_pdf.pages[0])

# Añadir índice
final_pdf.add_page(index_pdf.pages[0])

# Sobrescribir números de página y añadir PDFs
page_counter = 3  # Portada=1, índice=2
for page in all_pages:
    # Crear overlay con número de página
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    width, height = A4

    rect_width = 50
    rect_height = 20
    x_pos = width - 40 - rect_width/2
    y_pos = 30 - rect_height/2 - offset_y
    can.setFillColorRGB(1, 1, 1)
    can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
    can.setFillColorRGB(0, 0, 0)
    can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
    can.save()

    packet.seek(0)
    overlay_pdf = PdfReader(packet)
    overlay_page = overlay_pdf.pages[0]
    page.merge_page(overlay_page)

    final_pdf.add_page(page)
    page_counter += 1

# Guardar PDF final
with open(output_path, "wb") as f:
    final_pdf.write(f)

print(f"PDF final creado con portada e índice técnico: {output_path}")


PDF final creado con portada e índice técnico: /home/pk/Desktop/vatiaco/public/routers/PDF_final_con_portada_indice.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from io import BytesIO
from datetime import datetime

# -------- CONFIGURACIÓN --------
folder_path = "/home/pk/Desktop/frontend/public/routers"
output_path = os.path.join(folder_path, "PDF_final_con_portada_indice.pdf")
offset_y = 14.17  # 0.5 cm en puntos

# -------- RECOPILAR PDFS --------
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))
pdf_files.sort()

# -------- PORTADA --------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=A4)
width, height = A4

can.setFont("Helvetica-Bold", 24)
can.drawCentredString(width/2, height/2 + 50, "REPORTE DE DOCUMENTOS PDF")
can.setFont("Helvetica", 16)
can.drawCentredString(width/2, height/2, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
can.showPage()
can.save()

packet.seek(0)
cover_pdf = PdfReader(packet)

# -------- CREAR INDICE COMO TABLA --------
styles = getSampleStyleSheet()
styleN = styles['Normal']

index_data = []
page_counter = 2  # Portada = 1
all_pages = []

for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    index_data.append((os.path.relpath(pdf_path, folder_path), page_counter))
    for page in reader.pages:
        all_pages.append(page)
        page_counter += 1

# Crear PDF de índice usando Platypus
packet = BytesIO()
doc = SimpleDocTemplate(packet, pagesize=A4, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
elements = []

# Encabezado
elements.append(Paragraph("ÍNDICE DE DOCUMENTOS", styles['Title']))
elements.append(Spacer(1, 20))

# Tabla de índice
table_data = [["Capítulo", "Página"]]
for name, page_no in index_data:
    table_data.append([Paragraph(name, styleN), str(page_no)])

usable_width = A4[0] - doc.leftMargin - doc.rightMargin
col_widths = [usable_width * 0.85, usable_width * 0.15]

table = Table(table_data, colWidths=col_widths, hAlign='LEFT')
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),
    ('ALIGN',(0,0),(-1,-1),'LEFT'),
    ('VALIGN',(0,0),(-1,-1),'TOP'),
    ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
    ('BOX', (0,0), (-1,-1), 0.25, colors.black),
    ('FONTSIZE', (0,0), (-1,-1), 12),
    ('LEFTPADDING', (0,0), (-1,-1), 5),
    ('RIGHTPADDING', (0,0), (-1,-1), 5),
    ('TOPPADDING', (0,0), (-1,-1), 3),
    ('BOTTOMPADDING', (0,0), (-1,-1), 3),
]))
elements.append(table)
doc.build(elements)

packet.seek(0)
index_pdf = PdfReader(packet)

# -------- CREAR PDF FINAL --------
final_pdf = PdfWriter()

# Añadir portada
final_pdf.add_page(cover_pdf.pages[0])

# Añadir índice
final_pdf.add_page(index_pdf.pages[0])

# Sobrescribir números de página y añadir PDFs
page_counter = 3  # Portada=1, índice=2
for page in all_pages:
    # Crear overlay con número de página
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    width, height = A4

    rect_width = 50
    rect_height = 20
    x_pos = width - 40 - rect_width/2
    y_pos = 30 - rect_height/2 - offset_y
    can.setFillColorRGB(1, 1, 1)
    can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
    can.setFillColorRGB(0, 0, 0)
    can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
    can.save()

    packet.seek(0)
    overlay_pdf = PdfReader(packet)
    overlay_page = overlay_pdf.pages[0]
    page.merge_page(overlay_page)

    final_pdf.add_page(page)
    page_counter += 1

# Guardar PDF final
with open(output_path, "wb") as f:
    final_pdf.write(f)

print(f"PDF final creado con portada e índice: {output_path}")


PDF final creado con portada e índice: /home/pk/Desktop/vatiaco/public/routers/PDF_final_con_portada_indice.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
from datetime import datetime

# Carpeta raíz
folder_path = "/home/pk/Desktop/frontend/public/routers"



# Recorrer recursivamente y obtener PDFs
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

pdf_files.sort()

final_pdf = PdfWriter()
index = []
offset_y = 14.17  # 0.5 cm

# -------- PORTADA --------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=letter)
width, height = letter

can.setFont("Helvetica-Bold", 24)
can.drawCentredString(width/2, height/2 + 50, "REPORTE DE DOCUMENTOS PDF")
can.setFont("Helvetica", 16)
can.drawCentredString(width/2, height/2, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
can.showPage()
can.save()

packet.seek(0)
cover_pdf = PdfReader(packet)
final_pdf.add_page(cover_pdf.pages[0])

# -------- RECORRER PDFs Y AGREGARLOS --------
page_counter = 2  # portada = 1

all_pages = []  # guardamos todas las páginas para renumerar después
for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    index.append((os.path.relpath(pdf_path, folder_path), page_counter))
    for page in reader.pages:
        # Crear overlay con número de página
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        width, height = letter
        
        rect_width = 50
        rect_height = 20
        x_pos = width - 40 - rect_width/2
        y_pos = 30 - rect_height/2 - offset_y
        can.setFillColorRGB(1, 1, 1)
        can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)
        can.setFillColorRGB(0, 0, 0)
        can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
        can.save()
        
        packet.seek(0)
        overlay_pdf = PdfReader(packet)
        overlay_page = overlay_pdf.pages[0]
        page.merge_page(overlay_page)
        
        all_pages.append(page)
        page_counter += 1

# -------- ÍNDICE --------
packet = BytesIO()
can = canvas.Canvas(packet, pagesize=letter)
width, height = letter

can.setFont("Helvetica-Bold", 20)
can.drawString(50, height - 50, "ÍNDICE DE DOCUMENTOS")

can.setFont("Helvetica", 12)
y_position = height - 80
for name, page_no in index:
    can.drawString(50, y_position, f"{name} .......... {page_no}")
    y_position -= 20
    if y_position < 50:
        can.showPage()
        y_position = height - 50

can.showPage()
can.save()

packet.seek(0)
index_pdf = PdfReader(packet)

# Añadir índice después de portada
final_pdf.add_page(index_pdf.pages[0])

# Añadir todas las páginas de los PDFs
for page in all_pages:
    final_pdf.add_page(page)

# Guardar PDF final
output_path = os.path.join(folder_path, "PDF_final_con_portada_indice.pdf")
with open(output_path, "wb") as f:
    final_pdf.write(f)

print(f"PDF final creado con portada e índice: {output_path}")


PDF final creado con portada e índice: /home/pk/Desktop/vatiaco/public/routers/PDF_final_con_portada_indice.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

# Carpeta raíz
folder_path = "/home/pk/Desktop/frontend/public/routers"

# Recorrer recursivamente y obtener PDFs
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Ordenar alfabéticamente
pdf_files.sort()

# Contador global de páginas
page_counter = 1

# PDF final
final_pdf = PdfWriter()

for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    for page_num, page in enumerate(reader.pages, start=1):
        # Crear overlay con reportlab
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        width, height = letter
        
        # Ajuste para bajar 0.5 cm
        offset_y = 14.17  # 0.5 cm en puntos

        # Dibujar un rectángulo blanco sobre el número antiguo
        rect_width = 50
        rect_height = 20
        x_pos = width - 40 - rect_width/2
        y_pos = 30 - rect_height/2 - offset_y   # bajamos 0.5 cm
        can.setFillColorRGB(1, 1, 1)  # blanco
        can.rect(x_pos, y_pos, rect_width, rect_height, fill=1, stroke=0)

        # Número de página encima
        can.setFillColorRGB(0, 0, 0)  # negro
        can.drawRightString(width - 40, 30 - offset_y, str(page_counter))
        can.save()
        
        # Mover al inicio del BytesIO
        packet.seek(0)
        overlay_pdf = PdfReader(packet)
        overlay_page = overlay_pdf.pages[0]
        
        # Superponer overlay sobre la página original
        page.merge_page(overlay_page)
        
        # Agregar al PDF final
        final_pdf.add_page(page)
        page_counter += 1

# Guardar PDF final
output_path = os.path.join(folder_path, "PDF_unido_numerado.pdf")
with open(output_path, "wb") as f:
    final_pdf.write(f)

print(f"PDF unido y renumerado creado: {output_path}")


PDF unido y renumerado creado: /home/pk/Desktop/vatiaco/public/routers/PDF_unido_numerado.pdf


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter, PdfMerger
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

# Carpeta raíz
folder_path = "/home/pk/Desktop/frontend/public/routers"

# Recorrer recursivamente y obtener PDFs
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Ordenar alfabéticamente
pdf_files.sort()

# Contador global de páginas
page_counter = 1

# PDF final
final_pdf = PdfWriter()

for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    for page_num, page in enumerate(reader.pages, start=1):
        # Crear overlay con reportlab
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        width, height = letter
        # Número de página en esquina inferior derecha
        can.drawRightString(width - 40, 30, str(page_counter))
        can.save()
        
        # Mover al inicio del BytesIO
        packet.seek(0)
        overlay_pdf = PdfReader(packet)
        overlay_page = overlay_pdf.pages[0]
        
        # Superponer overlay sobre la página original
        page.merge_page(overlay_page)
        
        # Agregar al PDF final
        final_pdf.add_page(page)
        page_counter += 1

# Guardar PDF final
output_path = os.path.join(folder_path, "PDF_unido_numerado.pdf")
with open(output_path, "wb") as f:
    final_pdf.write(f)

print(f"PDF unido y renumerado creado: {output_path}")


PDF unido y renumerado creado: /home/pk/Desktop/vatiaco/public/routers/PDF_unido_numerado.pdf


In [ ]:
import os
from PyPDF2 import PdfMerger

# Carpeta raíz
folder_path = "/home/pk/Desktop/frontend/public/routers"

# Recorrer recursivamente y obtener PDFs
pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Ordenar alfabéticamente
pdf_files.sort()

# Crear un PDF final uniendo todos
merger = PdfMerger()

for pdf in pdf_files:
    merger.append(pdf)

# Guardar PDF final
output_path = os.path.join(folder_path, "PDF_unido.pdf")
merger.write(output_path)
merger.close()

print(f"PDF unido creado con éxito: {output_path}")


PDF unido creado con éxito: /home/pk/Desktop/vatiaco/public/routers/PDF_unido.pdf


In [ ]:
import pandas as pd
import os
from fpdf import FPDF
from datetime import datetime

# Carpeta raíz donde buscar archivos
folder_path = "/home/pk/Desktop/frontend/public/routers"

# Recorrer la carpeta recursivamente y tomar CSV y Excel
files = []
for root, dirs, filenames in os.walk(folder_path):
    for f in filenames:
        if f.endswith((".csv", ".xlsx", ".xls")):
            files.append(os.path.join(root, f))

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# -------- PORTADA --------
pdf.add_page()
pdf.set_font("Arial", 'B', 24)
pdf.ln(60)  # bajar un poco el texto
pdf.cell(0, 10, "REPORTE DE DATAFRAMES", ln=True, align='C')

pdf.set_font("Arial", '', 16)
pdf.ln(10)
pdf.cell(0, 10, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True, align='C')

pdf.add_page()  # Página del índice

# -------- ÍNDICE --------
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "ÍNDICE", ln=True)
pdf.add_page()  # Página donde empiezan los DataFrames

index = []

# -------- DATOS --------
for file in files:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
    else:
        df = pd.read_excel(file)

    # Reemplazar caracteres especiales por ASCII
    df = df.applymap(lambda x: str(x).encode('latin-1', errors='replace').decode('latin-1'))


    filename = os.path.relpath(file, folder_path)
    
    index.append((filename, pdf.page_no()))
    
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, filename, ln=True)
    
    pdf.set_font("Arial", '', 10)
    data = [df.columns.tolist()] + df.values.tolist()
    
    col_width = pdf.w / (len(df.columns) + 1)
    
    for row in data:
        for item in row:
            pdf.multi_cell(col_width, 8, str(item), border=1, ln=3, max_line_height=pdf.font_size)
        pdf.ln()
    
    pdf.add_page()

# -------- VOLVER AL ÍNDICE --------
pdf.page = 2  # la segunda página es el índice
pdf.set_font("Arial", '', 12)
pdf.ln(10)
for name, page in index:
    pdf.cell(0, 10, f"{name} ........ {page}", ln=True)

pdf.output("output.pdf")
print("PDF con portada creado con éxito.")


PDF con portada creado con éxito.


/tmp/ipykernel_25641/3775261229.py:21: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 24)
/tmp/ipykernel_25641/3775261229.py:23: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "REPORTE DE DATAFRAMES", ln=True, align='C')
/tmp/ipykernel_25641/3775261229.py:25: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", '', 16)
/tmp/ipykernel_25641/3775261229.py:27: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Generado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True, align='C')
/tmp/ipykernel_25641/3775261229.py:32: DeprecationWarning: Substituting font arial by core font helvetica 

In [1]:
from gtts import gTTS
from pydub import AudioSegment
import io

# SRT de prueba en memoria
subtitulos = [
    {"start": 0, "end": 2000, "text": "Hola, esto es una prueba."},
    {"start": 2500, "end": 5000, "text": "Estamos generando audio desde SRT."},
    {"start": 5500, "end": 8000, "text": "Todo se hará en memoria."}
]

# Lista para guardar audios de cada línea
audios = []

for sub in subtitulos:
    # Generar TTS
    tts = gTTS(text=sub["text"], lang="es")
    fp = io.BytesIO()
    tts.write_to_fp(fp)
    fp.seek(0)
    
    # Cargar en pydub
    audio = AudioSegment.from_file(fp, format="mp3")
    
    # Añadir silencio según el tiempo de inicio
    if sub["start"] > 0:
        audio = AudioSegment.silent(duration=sub["start"]) + audio
    audios.append(audio)

# Concatenar todos los audios
final_audio = sum(audios)

# Exportar audio final
final_audio.export("output_prueba.mp3", format="mp3")
print("Audio generado: output_prueba.mp3")


Audio generado: output_prueba.mp3
